In [1]:
import pandas as pd
import calendar
import holidays

In [2]:
# read csv files
df_bike = pd.read_csv("../bike_data/bike_data.csv", parse_dates=["Datetime"], index_col="Datetime")
df_bike.index = pd.to_datetime(df_bike.index, utc=True)
df_bike.index = df_bike.index.tz_convert('Europe/Berlin')

df_weather = pd.read_csv("../karlsruhe_weather/historical_weather_karslruhe.csv", parse_dates=["Datetime"], index_col="Datetime")
df_weather.index = pd.to_datetime(df_weather.index, utc=True)
df_weather.index = df_weather.index.tz_convert('Europe/Berlin')

df_weather_forecast = pd.read_csv("../karlsruhe_weather/weather_forecast_karlsruhe.csv", parse_dates=["Datetime"], index_col="Datetime")
df_weather_forecast.index = pd.to_datetime (df_weather_forecast.index, utc=True)
df_weather_forecast.index = df_weather_forecast.index.tz_convert('Europe/Berlin')

In [3]:
cutoff_date = pd.to_datetime("2024-02-22")

df_past = df_weather.loc[:cutoff_date]

df_future = df_weather_forecast.loc[cutoff_date + pd.Timedelta(seconds=1):]

df_combined = pd.concat([df_past, df_future])

C:\Users\Tim Flaschel\AppData\Local\Temp\ipykernel_21120\892509548.py:3: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  df_past = df_weather.loc[:cutoff_date]
C:\Users\Tim Flaschel\AppData\Local\Temp\ipykernel_21120\892509548.py:5: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  df_future = df_weather_forecast.loc[cutoff_date + pd.Timedelta(seconds=1):]


In [4]:
df = df_bike.combine_first(df_combined)

#  Add Baden Württemberg holiday information to the dataframe
bw_feiertage = holidays.Germany(years=range(2012, 2026), state='BW')
df['public_holiday'] = df.index.to_series().apply(lambda x: x in bw_feiertage)
df['public_holiday'] = df['public_holiday'].astype(int)

In [10]:
df = df.loc["2013-01-01":"2025-03-01"]

In [11]:
# Start- und Enddatum im DataFrame-Index bestimmen
start_date = df.index.min()
end_date = df.index.max()

# Alle Stunden zwischen Start- und Enddatum erzeugen
all_hours = pd.date_range(start=start_date, end=end_date, freq='H')

# Fehlende Stunden finden
missing_hours = all_hours.difference(df.index)

# Anzahl der fehlenden Stunden
missing_count = len(missing_hours)

print(f"Es fehlen {missing_count} Stunden im Index.")
print("Liste der fehlenden Stunden:")
print(missing_hours)

nan_count = df['target'].isna().sum()
print(f"Anzahl der NaN-Werte in der target-Spalte: {nan_count}")

Es fehlen 0 Stunden im Index.
Liste der fehlenden Stunden:
DatetimeIndex([], dtype='datetime64[ns, Europe/Berlin]', freq='H')
Anzahl der NaN-Werte in der target-Spalte: 0


In [12]:
df['target'] = pd.to_numeric(df['target'], errors='coerce')

C:\Users\Tim Flaschel\AppData\Local\Temp\ipykernel_21120\506237507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = pd.to_numeric(df['target'], errors='coerce')


In [13]:
df.to_csv("bike_data_weather_forecast.csv", index=True)

In [14]:
df

,cloud_cover,precipitation,relative_humidity_2m,shortwave_radiation,target,temperature_2m,weather_code,wind_speed_10m,public_holiday
Datetime,,,,,,,,,
2013-01-01 00:00:00+01:00,95.0,0.0,55.362625,0.0,30.0,6.3625,3.0,13.551500,1
2013-01-01 01:00:00+01:00,99.0,0.0,54.982826,0.0,19.0,6.4625,3.0,13.104197,1
2013-01-01 02:00:00+01:00,100.0,0.0,54.834316,0.0,33.0,6.6625,3.0,14.332340,1
2013-01-01 03:00:00+01:00,99.0,0.0,56.056946,0.0,31.0,6.6625,3.0,15.778518,1
2013-01-01 04:00:00+01:00,100.0,0.0,57.355602,0.0,40.0,6.8625,3.0,17.102840,1
...,...,...,...,...,...,...,...,...,...
2025-03-01 19:00:00+01:00,80.0,0.0,80.000000,0.0,224.0,4.7805,2.0,1.972308,0
2025-03-01 20:00:00+01:00,77.0,0.0,84.000000,0.0,159.0,4.0305,2.0,1.664332,0
2025-03-01 21:00:00+01:00,33.0,0.0,88.000000,0.0,94.0,3.3805,1.0,1.749286,0
